<a href="https://colab.research.google.com/github/lawrennd/fitkit/blob/main/examples/spectral_entropic_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spectral-Entropic Comparison: Demonstrating Different Network Morphologies

This notebook demonstrates the different plotting modalities described in the economic fitness paper,
comparing spectral measures (ECI/PCI) with entropic measures (Fitness/Complexity) across various
synthetic network structures.

## Key Concepts

- **ECI (Economic Complexity Index)**: Spectral measure using second eigenvector of degree-normalized random walk
- **Fitness**: Entropic measure from maximum-entropy Sinkhorn scaling
- **PCI (Product Complexity Index)**: Spectral measure for products
- **Complexity**: Entropic measure for products

## Four Network Morphologies

1. **Morphology A**: Single nested hierarchy (tight monotone trend)
2. **Morphology B**: Low-conductance communities (separated lobes)
3. **Morphology C**: Core-periphery structure (banana/curved trend)
4. **Morphology D**: Multi-scale structure (diffuse cloud)

In [ ]:
import sys
import subprocess
from pathlib import Path


def _pip_install(args: list[str]) -> None:
    cmd = [sys.executable, "-m", "pip", *args]
    print("Running:", " ".join(cmd))
    subprocess.check_call(cmd)


def ensure_fitkit_installed() -> None:
    """Prefer editable local install; fall back to GitHub.

    - Local (typical): `pip install -e ..` when running from `examples/`
    - Colab/remote: `pip install git+https://github.com/lawrennd/fitkit.git`
    """
    try:
        import fitkit  # noqa: F401

        return
    except ImportError:
        pass

    here = Path.cwd().resolve()
    candidates = [here, here.parent, here.parent.parent]

    for root in candidates:
        if (root / "pyproject.toml").exists() and (root / "fitkit").is_dir():
            _pip_install(["install", "-e", str(root)])
            return

    _pip_install(["install", "git+https://github.com/lawrennd/fitkit.git"])


ensure_fitkit_installed()
import fitkit

print("fitkit version:", getattr(fitkit, "__version__", "unknown"))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import sparse
from scipy.stats import pearsonr, spearmanr
from sklearn.cluster import KMeans

# Import scikit-learn style estimators from fitkit
from fitkit.algorithms import FitnessComplexity, ECI

# Import community analysis helpers
from community_analysis_helpers import (
    detect_communities_from_eigenvectors,
    analyze_within_communities
)

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['font.size'] = 10

## Helper Functions

In [ ]:
def compute_eigenvalues_for_diagnostics(M, n_eigs=10):
    """
    Compute eigenvalues of the degree-normalized transition matrices for diagnostic purposes.
    Used for spectral gap analysis alongside the ECI class.
    """
    from scipy.sparse.linalg import eigs

    M = M.astype(float)
    n_countries, n_products = M.shape

    k_c = M.sum(axis=1) + 1e-10
    k_p = M.sum(axis=0) + 1e-10

    D_c_inv = sparse.diags(1.0 / k_c)
    D_p_inv = sparse.diags(1.0 / k_p)
    M_sparse = sparse.csr_matrix(M)

    T_countries = D_c_inv @ M_sparse @ D_p_inv @ M_sparse.T
    T_products = D_p_inv @ M_sparse.T @ D_c_inv @ M_sparse

    n_eigs_c = min(n_eigs, n_countries - 1)
    n_eigs_p = min(n_eigs, n_products - 1)

    if n_countries > 2:
        eigenvalues_c, _ = eigs(T_countries, k=n_eigs_c, which='LM')
        eigenvalues_c = np.sort(np.real(eigenvalues_c))[::-1]
    else:
        eigenvalues_c = np.array([1.0, 0.0])

    if n_products > 2:
        eigenvalues_p, _ = eigs(T_products, k=n_eigs_p, which='LM')
        eigenvalues_p = np.sort(np.real(eigenvalues_p))[::-1]
    else:
        eigenvalues_p = np.array([1.0, 0.0])

    return eigenvalues_c, eigenvalues_p


def standardize(x):
    """Standardize to zero mean, unit variance."""
    return (x - x.mean()) / (x.std() + 1e-10)


def compute_spectral_gap(eigenvalues):
    """
    Compute spectral gap ratio: (lambda_3 - lambda_2) / (1 - lambda_2)
    For Laplacian eigenvalues: lambda_L = 1 - lambda_T
    """
    if len(eigenvalues) < 3:
        return 0.0

    lambda_2 = eigenvalues[1]
    lambda_3 = eigenvalues[2]

    # Convert to Laplacian eigenvalues
    lambda_2_L = 1 - lambda_2
    lambda_3_L = 1 - lambda_3

    gap_ratio = lambda_3_L / (lambda_2_L + 1e-10)
    return gap_ratio


def compute_cheeger_conductance(M, eci):
    """
    Compute Cheeger conductance for the cut induced by ECI (split by sign).

    Phi = cut(S, S_bar) / vol(S)
    """
    # Partition countries by sign of ECI
    S = eci > 0
    S_bar = ~S

    if S.sum() == 0 or S_bar.sum() == 0:
        return 0.0

    # Compute diversification (degrees)
    k_c = M.sum(axis=1)

    # Volume of S
    vol_S = k_c[S].sum()
    vol_S_bar = k_c[S_bar].sum()

    # Count cut edges: countries in S connected to countries in S_bar via shared products
    # For bipartite graph: cut is measured via product connections
    M_S = M[S, :]
    M_S_bar = M[S_bar, :]

    # Products exported by S
    products_S = M_S.sum(axis=0) > 0
    # Products exported by S_bar
    products_S_bar = M_S_bar.sum(axis=0) > 0
    # Shared products
    shared_products = products_S & products_S_bar

    # Cut size: number of edges between S and S_bar
    cut_size = shared_products.sum()

    # Conductance
    conductance = cut_size / min(vol_S, vol_S_bar) if min(vol_S, vol_S_bar) > 0 else 0

    return conductance

## Generate Synthetic Networks

We'll create four types of networks corresponding to the morphologies described in the paper.

In [ ]:
def generate_nested_network(n_countries=50, n_products=80, seed=42):
    """
    Morphology A: Perfectly nested network.
    Countries ordered by capability, products by complexity.
    M[c,p] = 1 if capability[c] >= complexity[p]
    """
    np.random.seed(seed)

    # Generate capability and complexity scores
    capability = np.sort(np.random.uniform(0, 1, n_countries))
    complexity = np.sort(np.random.uniform(0, 1, n_products))

    # Create nested structure
    M = np.zeros((n_countries, n_products))
    for c in range(n_countries):
        for p in range(n_products):
            if capability[c] >= complexity[p]:
                # Add some noise
                if np.random.random() > 0.1:  # 90% correct nesting
                    M[c, p] = 1
            else:
                if np.random.random() < 0.05:  # 5% noise
                    M[c, p] = 1

    return M, "Nested Hierarchy"


def generate_modular_network(n_countries=50, n_products=80, n_communities=2, seed=42):
    """
    Morphology B: Modular network with low-conductance communities.
    Two separate communities with weak connections between them.
    """
    np.random.seed(seed)

    countries_per_comm = n_countries // n_communities
    products_per_comm = n_products // n_communities

    M = np.zeros((n_countries, n_products))

    for comm in range(n_communities):
        c_start = comm * countries_per_comm
        c_end = (comm + 1) * countries_per_comm if comm < n_communities - 1 else n_countries
        p_start = comm * products_per_comm
        p_end = (comm + 1) * products_per_comm if comm < n_communities - 1 else n_products

        # Generate nested structure within community
        comm_size_c = c_end - c_start
        comm_size_p = p_end - p_start

        capability = np.sort(np.random.uniform(0, 1, comm_size_c))
        complexity = np.sort(np.random.uniform(0, 1, comm_size_p))

        for i, c in enumerate(range(c_start, c_end)):
            for j, p in enumerate(range(p_start, p_end)):
                if capability[i] >= complexity[j]:
                    if np.random.random() > 0.1:
                        M[c, p] = 1
                else:
                    if np.random.random() < 0.05:
                        M[c, p] = 1

    # Add weak inter-community connections (low conductance)
    for c in range(n_countries):
        for p in range(n_products):
            c_comm = c // countries_per_comm
            p_comm = p // products_per_comm
            if c_comm != p_comm and np.random.random() < 0.03:  # 3% cross-community
                M[c, p] = 1

    return M, "Modular (Low Conductance)"


def generate_core_periphery_network(n_countries=50, n_products=80, core_fraction=0.3, seed=42):
    """
    Morphology C: Core-periphery network.
    Core countries are highly diversified, periphery countries are specialized.
    """
    np.random.seed(seed)

    n_core = int(n_countries * core_fraction)
    n_periphery = n_countries - n_core

    M = np.zeros((n_countries, n_products))

    # Core countries: export most products
    for c in range(n_core):
        n_products_export = int(np.random.uniform(0.7, 0.95) * n_products)
        products = np.random.choice(n_products, n_products_export, replace=False)
        M[c, products] = 1

    # Periphery countries: export few products
    for c in range(n_core, n_countries):
        n_products_export = int(np.random.uniform(0.05, 0.2) * n_products)
        products = np.random.choice(n_products, n_products_export, replace=False)
        M[c, products] = 1

    return M, "Core-Periphery"


def generate_multiscale_network(n_countries=50, n_products=80, n_clusters=4, seed=42):
    """
    Morphology D: Multi-scale network with overlapping communities.
    Multiple clusters with significant overlap.
    """
    np.random.seed(seed)

    M = np.zeros((n_countries, n_products))

    countries_per_cluster = n_countries // n_clusters
    products_per_cluster = n_products // n_clusters

    for cluster in range(n_clusters):
        # Each cluster has its own set of countries and products
        c_start = cluster * countries_per_cluster
        c_end = (cluster + 1) * countries_per_cluster if cluster < n_clusters - 1 else n_countries

        # Random product selection for this cluster
        cluster_products = np.random.choice(
            n_products,
            int(n_products * 0.4),  # 40% of products
            replace=False
        )

        for c in range(c_start, c_end):
            # Each country exports a subset of cluster products
            n_export = int(np.random.uniform(0.3, 0.8) * len(cluster_products))
            export_products = np.random.choice(cluster_products, n_export, replace=False)
            M[c, export_products] = 1

    # Add cross-cluster connections (overlaps)
    for c in range(n_countries):
        n_cross = int(np.random.uniform(0.1, 0.2) * n_products)
        cross_products = np.random.choice(n_products, n_cross, replace=False)
        M[c, cross_products] = 1

    return M, "Multi-Scale"

## Analysis and Plotting Function

In [ ]:
def analyze_and_plot(M, network_name):
    """
    Complete analysis of a network:
    1. Compute spectral measures (ECI, PCI)
    2. Compute entropic measures (Fitness, Complexity)
    3. Create spectral-entropic comparison plots
    4. Compute diagnostics (correlation, spectral gap, conductance)
    """
    print(f"\n{'='*60}")
    print(f"Analyzing: {network_name}")
    print(f"{'='*60}\n")

    # Convert to sparse matrix for sklearn-style estimators
    M_sparse = sparse.csr_matrix(M)

    # Compute spectral measures using sklearn-style ECI estimator
    eci_estimator = ECI()
    eci, pci = eci_estimator.fit_transform(M_sparse)

    # Compute entropic measures using sklearn-style FitnessComplexity estimator
    fc_estimator = FitnessComplexity(n_iter=200, tol=1e-10, verbose=False)
    fitness, complexity = fc_estimator.fit_transform(M_sparse)

    # Standardize fitness and complexity for comparison
    fitness_std = standardize(fitness)
    complexity_std = standardize(complexity)

    # Compute eigenvalues for diagnostic purposes
    eigenvalues_c, eigenvalues_p = compute_eigenvalues_for_diagnostics(M)

    # Compute correlations
    pearson_countries, _ = pearsonr(eci, fitness_std)
    spearman_countries, _ = spearmanr(eci, fitness_std)
    pearson_products, _ = pearsonr(pci, complexity_std)
    spearman_products, _ = spearmanr(pci, complexity_std)

    # Compute spectral gaps
    gap_ratio_c = compute_spectral_gap(eigenvalues_c)
    gap_ratio_p = compute_spectral_gap(eigenvalues_p)

    # Compute Cheeger conductance
    conductance = compute_cheeger_conductance(M, eci)

    # Print diagnostics
    print("Correlation Analysis:")
    print(f"  Countries (ECI vs Fitness):")
    print(f"    Pearson r  = {pearson_countries:.3f}")
    print(f"    Spearman ρ = {spearman_countries:.3f}")
    print(f"  Products (PCI vs Complexity):")
    print(f"    Pearson r  = {pearson_products:.3f}")
    print(f"    Spearman ρ = {spearman_products:.3f}")

    print(f"\nSpectral Gap Analysis:")
    print(f"  Countries: λ₃ᴸ / λ₂ᴸ = {gap_ratio_c:.3f}")
    print(f"  Products:  λ₃ᴸ / λ₂ᴸ = {gap_ratio_p:.3f}")
    print(f"  (Large ratio >2-3 indicates 1D structure)")

    print(f"\nCheeger Conductance: {conductance:.3f}")
    print(f"  (Low conductance <0.1 indicates community structure)")

    print(f"\nTop 5 Eigenvalues (countries): {eigenvalues_c[:5]}")
    print(f"Top 5 Eigenvalues (products):  {eigenvalues_p[:5]}")

    # Create plots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(f'{network_name}: Spectral-Entropic Analysis', fontsize=16, fontweight='bold')

    # 1. Network visualization (adjacency matrix)
    ax = axes[0, 0]
    im = ax.imshow(M, aspect='auto', cmap='Blues', interpolation='nearest')
    ax.set_title('Network Structure\n(Adjacency Matrix)')
    ax.set_xlabel('Products')
    ax.set_ylabel('Countries')
    plt.colorbar(im, ax=ax)

    # 2. Country spectral-entropic comparison
    ax = axes[0, 1]
    ax.scatter(eci, fitness_std, alpha=0.6, s=50)
    ax.set_xlabel('ECI (Spectral)', fontweight='bold')
    ax.set_ylabel('Fitness (Entropic)', fontweight='bold')
    ax.set_title(f'Country Comparison\nr={pearson_countries:.3f}, ρ={spearman_countries:.3f}')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax.axvline(x=0, color='k', linestyle='--', alpha=0.3)

    # Add regression line
    if len(eci) > 1:
        z = np.polyfit(eci, fitness_std, 1)
        p = np.poly1d(z)
        x_line = np.linspace(eci.min(), eci.max(), 100)
        ax.plot(x_line, p(x_line), 'r--', alpha=0.5, linewidth=2)

    # 3. Product spectral-entropic comparison
    ax = axes[0, 2]
    ax.scatter(pci, complexity_std, alpha=0.6, s=50, color='green')
    ax.set_xlabel('PCI (Spectral)', fontweight='bold')
    ax.set_ylabel('Complexity (Entropic)', fontweight='bold')
    ax.set_title(f'Product Comparison\nr={pearson_products:.3f}, ρ={spearman_products:.3f}')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax.axvline(x=0, color='k', linestyle='--', alpha=0.3)

    # Add regression line
    if len(pci) > 1:
        z = np.polyfit(pci, complexity_std, 1)
        p = np.poly1d(z)
        x_line = np.linspace(pci.min(), pci.max(), 100)
        ax.plot(x_line, p(x_line), 'r--', alpha=0.5, linewidth=2)

    # 4. Country rankings comparison
    ax = axes[1, 0]
    country_ranks_eci = np.argsort(np.argsort(-eci))
    country_ranks_fitness = np.argsort(np.argsort(-fitness_std))
    ax.scatter(country_ranks_eci, country_ranks_fitness, alpha=0.6, s=50)
    ax.plot([0, len(eci)], [0, len(eci)], 'r--', alpha=0.5)
    ax.set_xlabel('ECI Rank')
    ax.set_ylabel('Fitness Rank')
    ax.set_title('Country Rank Comparison')
    ax.grid(True, alpha=0.3)

    # 5. Eigenvalue spectrum
    ax = axes[1, 1]
    n_show = min(10, len(eigenvalues_c))
    x = np.arange(1, n_show + 1)
    ax.plot(x, eigenvalues_c[:n_show], 'o-', label='Countries', linewidth=2, markersize=8)
    ax.plot(x, eigenvalues_p[:n_show], 's-', label='Products', linewidth=2, markersize=8)
    ax.set_xlabel('Eigenvalue Index')
    ax.set_ylabel('Eigenvalue (Transition Matrix)')
    ax.set_title('Eigenvalue Spectrum\n(Spectral Gap Analysis)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axhline(y=1, color='k', linestyle='--', alpha=0.3)

    # Highlight spectral gap
    if n_show >= 3:
        ax.axvspan(1.5, 2.5, alpha=0.2, color='yellow', label='Gap region')

    # 6. Distribution comparison
    ax = axes[1, 2]
    ax.hist(eci, bins=20, alpha=0.5, label='ECI', density=True)
    ax.hist(fitness_std, bins=20, alpha=0.5, label='Fitness', density=True)
    ax.set_xlabel('Standardized Value')
    ax.set_ylabel('Density')
    ax.set_title('Distribution Comparison\n(Countries)')
    ax.legend()
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Interpret morphology (data-driven, less prescriptive)
    print(f"\nMorphology Interpretation:")
    print(f"  (Note: These are qualitative patterns, not rigid thresholds)\n")
    
    # Describe what we observe
    print(f"  Correlation: {pearson_countries:.3f}")
    if pearson_countries > 0.85:
        print("    → Very high: likely tight monotone trend (Morphology A pattern)")
    elif pearson_countries > 0.6:
        print("    → Moderate to high: some alignment with possible structure")
    else:
        print("    → Low: methods capture different aspects of network structure")
    
    print(f"\n  Spectral gap λ₃ᴸ/λ₂ᴸ: {gap_ratio_c:.3f}")
    if gap_ratio_c > 3:
        print("    → Large gap: dominant 1D mode (single gradient)")
    elif gap_ratio_c > 1.5:
        print("    → Moderate gap: primary mode present but not overwhelming")
    else:
        print("    → Small gap: multiple comparable modes (Morphology D pattern)")
    
    print(f"\n  Conductance: {conductance:.3f}")
    if conductance < 0.2:
        print("    → Low: potential community structure (Morphology B pattern)")
        print("    → Consider community detection for within-group analysis")
    else:
        print("    → Moderate to high: well-connected network")
    
    print(f"\n  Visual inspection of scatter plot is crucial!")
    print(f"  Look for: tight trend, separated lobes, curvature, or diffuse cloud\n")

    return {
        'eci': eci,
        'pci': pci,
        'fitness': fitness_std,
        'complexity': complexity_std,
        'pearson_countries': pearson_countries,
        'spearman_countries': spearman_countries,
        'pearson_products': pearson_products,
        'spearman_products': spearman_products,
        'gap_ratio_c': gap_ratio_c,
        'gap_ratio_p': gap_ratio_p,
        'conductance': conductance,
        'eigenvalues_c': eigenvalues_c,
        'eigenvalues_p': eigenvalues_p
    }

## Morphology A: Nested Hierarchy

Expected characteristics:
- High correlation (r > 0.85) between ECI and Fitness
- Large spectral gap (λ₃ᴸ/λ₂ᴸ > 2-3)
- Tight monotone trend in scatter plots
- Both methods capture the same capability gradient

In [ ]:
M_nested, name_nested = generate_nested_network(n_countries=50, n_products=80)
results_nested = analyze_and_plot(M_nested, name_nested)

## Morphology B: Modular Network (Low-Conductance Communities)

Expected characteristics:
- Separated lobes in scatter plot
- Low global correlation (r < 0.6)
- Low Cheeger conductance (Φ < 0.1)
- ECI acts as community indicator
- Fitness measures within-community hierarchy

In [ ]:
M_modular, name_modular = generate_modular_network(n_countries=50, n_products=80, n_communities=2)
results_modular = analyze_and_plot(M_modular, name_modular)

## Morphology C: Core-Periphery Structure

Expected characteristics:
- Banana or curved trend in scatter plot
- Moderate correlation (0.5 < r < 0.85)
- ECI distinguishes core from periphery linearly
- Fitness compresses periphery values via harmonic aggregation

In [ ]:
M_core_periphery, name_core_periphery = generate_core_periphery_network(n_countries=50, n_products=80)
results_core_periphery = analyze_and_plot(M_core_periphery, name_core_periphery)

## Morphology D: Multi-Scale Structure

Expected characteristics:
- Diffuse, fan-shaped or smeared cloud
- Weak correlation (r < 0.5)
- Small spectral gap (λ₃ᴸ/λ₂ᴸ < 2)
- Multiple comparable eigenvalues
- Single ECI coordinate insufficient

In [ ]:
M_multiscale, name_multiscale = generate_multiscale_network(n_countries=50, n_products=80, n_clusters=4)
results_multiscale = analyze_and_plot(M_multiscale, name_multiscale)

## Summary Comparison Across All Morphologies

In [ ]:
import pandas as pd

# Create summary table
summary_data = {
    'Morphology': ['A: Nested', 'B: Modular', 'C: Core-Periphery', 'D: Multi-Scale'],
    'Pearson r (Countries)': [
        results_nested['pearson_countries'],
        results_modular['pearson_countries'],
        results_core_periphery['pearson_countries'],
        results_multiscale['pearson_countries']
    ],
    'Spectral Gap (λ₃/λ₂)': [
        results_nested['gap_ratio_c'],
        results_modular['gap_ratio_c'],
        results_core_periphery['gap_ratio_c'],
        results_multiscale['gap_ratio_c']
    ],
    'Conductance Φ': [
        results_nested['conductance'],
        results_modular['conductance'],
        results_core_periphery['conductance'],
        results_multiscale['conductance']
    ],
    'λ₂ (Countries)': [
        results_nested['eigenvalues_c'][1],
        results_modular['eigenvalues_c'][1],
        results_core_periphery['eigenvalues_c'][1],
        results_multiscale['eigenvalues_c'][1]
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.round(3)

print("\n" + "="*80)
print("SUMMARY: Diagnostic Comparison Across Network Morphologies")
print("="*80 + "\n")
print(summary_df.to_string(index=False))
print("\n" + "="*80)

print("\nDiagnostic Patterns (Qualitative Guide):")
print("-" * 80)
print("Correlation (r):")
print("  • Very high (>0.85): Methods agree - likely nested hierarchy")
print("  • Moderate (0.5-0.85): Partial agreement - inspect visually for curvature")
print("  • Low (<0.5): Methods diverge - richer structure or multiple scales")
print("\nSpectral Gap (λ₃ᴸ/λ₂ᴸ):")
print("  • Large (>3): Strong single dominant mode")
print("  • Moderate (1.5-3): Primary mode present")
print("  • Small (<1.5): Multiple comparable modes - use higher eigenvectors")
print("\nConductance (Φ):")
print("  • Low (<0.2): Potential communities - try community detection")
print("  • Moderate to high (>0.2): Well-connected")
print("\nIMPORTANT: These are patterns, not rules. Visual inspection is essential!")
print("="*80)

## Visualization: All Morphologies Side-by-Side

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
fig.suptitle('Spectral-Entropic Comparison Across Network Morphologies',
             fontsize=16, fontweight='bold')

all_results = [
    (results_nested, 'A: Nested', 'blue'),
    (results_modular, 'B: Modular', 'red'),
    (results_core_periphery, 'C: Core-Periphery', 'green'),
    (results_multiscale, 'D: Multi-Scale', 'purple')
]

# Row 1: Country comparisons
for i, (results, name, color) in enumerate(all_results):
    ax = axes[0, i]
    ax.scatter(results['eci'], results['fitness'], alpha=0.6, s=40, color=color)
    ax.set_xlabel('ECI (Spectral)')
    ax.set_ylabel('Fitness (Entropic)' if i == 0 else '')
    ax.set_title(f"{name}\nr={results['pearson_countries']:.2f}")
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.2)
    ax.axvline(x=0, color='k', linestyle='--', alpha=0.2)

    # Fit line
    if len(results['eci']) > 1:
        z = np.polyfit(results['eci'], results['fitness'], 1)
        p = np.poly1d(z)
        x_line = np.linspace(results['eci'].min(), results['eci'].max(), 100)
        ax.plot(x_line, p(x_line), 'k--', alpha=0.5, linewidth=2)

# Row 2: Product comparisons
for i, (results, name, color) in enumerate(all_results):
    ax = axes[1, i]
    ax.scatter(results['pci'], results['complexity'], alpha=0.6, s=40, color=color)
    ax.set_xlabel('PCI (Spectral)')
    ax.set_ylabel('Complexity (Entropic)' if i == 0 else '')
    ax.set_title(f"Products: r={results['pearson_products']:.2f}")
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.2)
    ax.axvline(x=0, color='k', linestyle='--', alpha=0.2)

    # Fit line
    if len(results['pci']) > 1:
        z = np.polyfit(results['pci'], results['complexity'], 1)
        p = np.poly1d(z)
        x_line = np.linspace(results['pci'].min(), results['pci'].max(), 100)
        ax.plot(x_line, p(x_line), 'k--', alpha=0.5, linewidth=2)

plt.tight_layout()
plt.show()

## Operational Diagnostic Workflow

Following the paper's recommendations, here's the systematic workflow for analyzing a new network:

1. **Inspect plot morphology**: Visually classify scatter plot (tight, multi-lobe, curved, diffuse)
2. **Compute concordance**: Calculate Pearson and Spearman correlations
3. **Assess spectral gap**: Examine λ₃ᴸ/λ₂ᴸ ratio
4. **Cheeger conductance test**: For suspected communities, compute conductance of cut
5. **Identify outliers**: Investigate strongly discordant countries/products
6. **Validate with domain knowledge**: Compare with known economic structure
7. **Choose appropriate measure(s)**:
   - Rankings and gradients → Use ECI
   - Bottleneck identification → Use Fitness-Complexity
   - Community structure → Use ECI for blocks, Fitness within blocks
   - Comprehensive understanding → Report both

## Community Detection and Within-Community Analysis

When sub-communities exist, we can use higher eigenvectors to detect and separate them,
then analyze ECI vs Fitness within each community independently.

This addresses the paper's discussion of:
- **Morphology B**: Global ECI acts as community indicator, Fitness measures within-community hierarchy
- Low global correlation can coexist with high within-community correlations
- Using higher eigenvectors (ψ₃, ψ₄, ...) for multi-scale structure

In [ ]:
def analyze_network_with_communities(M, network_name, eci, fitness_std, eigenvalues_c):
    """
    Enhanced analysis that detects communities and analyzes them separately.
    """
    print(f"\n{'─'*60}")
    print(f"Community Detection Analysis: {network_name}")
    print(f"{'─'*60}\n")
    
    # Detect communities using eigenvector clustering
    community_labels, n_communities = detect_communities_from_eigenvectors(M, n_communities='auto')
    
    print(f"Detected {n_communities} communities")
    
    if n_communities == 1:
        print("  → Single cohesive network (no sub-communities detected)")
        return None
    
    # Show community sizes
    unique_labels, counts = np.unique(community_labels, return_counts=True)
    print(f"\nCommunity sizes:")
    for label, count in zip(unique_labels, counts):
        print(f"  Community {label}: {count} countries ({count/len(community_labels)*100:.1f}%)")
    
    # Analyze within each community
    community_stats = analyze_within_communities(M, community_labels, ECI, FitnessComplexity)
    
    if community_stats is None:
        print("\n  → Could not perform within-community analysis")
        return None
    
    # Compare global vs within-community correlations
    global_r = pearsonr(eci[~np.isnan(eci)], fitness_std[~np.isnan(eci)])[0]
    
    print(f"\n{'Correlation Comparison (Global vs Within-Community)':^60}")
    print(f"{'─'*60}")
    print(f"{'Global correlation:':<30} r = {global_r:6.3f}")
    print(f"\n{'Within-community correlations:':<30}")
    
    for stat in community_stats:
        print(f"  Community {stat['community_id']} (n={stat['n_members']:2d}):  r = {stat['correlation']:6.3f} (p={stat['p_value']:.3e})")
    
    avg_within = np.mean([s['correlation'] for s in community_stats])
    print(f"\n{'Average within-community:':<30} r = {avg_within:6.3f}")
    
    # Visualize communities in spectral-entropic space
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f'{network_name}: Community Structure in Spectral-Entropic Space', 
                 fontsize=14, fontweight='bold')
    
    # Plot 1: Communities colored in ECI-Fitness space
    ax = axes[0]
    colors = plt.cm.tab10(np.linspace(0, 1, n_communities))
    for comm_id in range(n_communities):
        mask = community_labels == comm_id
        ax.scatter(eci[mask], fitness_std[mask], 
                  c=[colors[comm_id]], label=f'Community {comm_id}',
                  alpha=0.7, s=60, edgecolors='black', linewidths=0.5)
    
    ax.set_xlabel('ECI (Spectral)', fontweight='bold')
    ax.set_ylabel('Fitness (Entropic)', fontweight='bold')
    ax.set_title(f'Communities Detected\\nGlobal r={global_r:.3f}')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.2)
    ax.axvline(x=0, color='k', linestyle='--', alpha=0.2)
    
    # Plot 2: Eigenvalue spectrum with gaps marked
    ax = axes[1]
    eigenvalues_L = 1 - eigenvalues_c
    n_show = min(8, len(eigenvalues_L))
    x = np.arange(1, n_show + 1)
    ax.plot(x, eigenvalues_L[:n_show], 'o-', linewidth=2, markersize=10, color='steelblue')
    
    # Mark the eigengaps
    if n_show >= 3:
        for i in range(1, n_show - 1):
            gap = eigenvalues_L[i+1] - eigenvalues_L[i]
            ax.axvspan(i + 0.5, i + 1.5, alpha=0.15, color='yellow')
            ax.text(i + 1, eigenvalues_L[i] + gap/2, f'Δ={gap:.3f}', 
                   ha='center', fontsize=8, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    ax.set_xlabel('Eigenvalue Index i')
    ax.set_ylabel('Laplacian Eigenvalue λᴸᵢ = 1 - λᵢᵀ')
    ax.set_title('Eigenvalue Spectrum with Gaps\n(Eigengap heuristic for community detection)')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Interpretation
    print(f"\n{'Key Insight:':^60}")
    print(f"{'─'*60}")
    if avg_within > global_r + 0.2:
        print("✓ Within-community correlations are MUCH HIGHER than global")
        print("  → ECI identifies community boundaries (block labels)")
        print("  → Fitness measures capability within each community")
        print("  → This is Morphology B: complementary perspectives!")
    elif avg_within > global_r + 0.1:
        print("✓ Within-community correlations are somewhat higher than global")
        print("  → Some community structure present")
        print("  → Both methods partially capture different aspects")
    else:
        print("→ Global and within-community correlations are similar")
        print("  → Detected 'communities' may not be meaningful")
        print("  → Network may have other structure (core-periphery, multi-scale)")
    
    return community_stats

In [ ]:
# Demonstrate community analysis on the modular network
if 'results_modular' in dir():
    analyze_network_with_communities(
        M_modular, 
        name_modular, 
        results_modular['eci'],
        results_modular['fitness'],
        results_modular['eigenvalues_c']
    )

## Conclusion

This notebook demonstrates the four main network morphologies described in the paper:

**Morphology A (Nested Hierarchy)**: High correlation, large spectral gap, tight monotone trend. Both ECI and Fitness capture the same capability gradient.

**Morphology B (Modular)**: Low global correlation, low conductance, separated lobes. ECI identifies communities while Fitness ranks within communities.

**Morphology C (Core-Periphery)**: Moderate correlation, curved trend. Harmonic aggregation compresses periphery values.

**Morphology D (Multi-Scale)**: Weak correlation, small spectral gap, diffuse cloud. Single ECI coordinate insufficient; need higher eigenvectors.

### Key Takeaways

1. **Complementarity**: ECI and Fitness-Complexity are complementary, not competing methods
2. **Spectral gap**: Large gap validates 1D embedding; small gap indicates multi-scale structure
3. **Discordance is informative**: Disagreement between methods reveals richer structure
4. **Diagnostic workflow**: Systematic analysis of correlations, gaps, and conductance guides interpretation
5. **Context matters**: Choose appropriate measure based on research question and network structure